In [1]:
from quantileRegression import quantileRegression
import numpy as np

Welcome to ROOTaaS 6.06/02


In [2]:
# Variable you want to correct
ylist = ["R9", "S4", "SigmaIeIe", "EtaWidth", "PhiWidth", "CovarianceIphiIphi", "SigmaRR"]

# Input variables used for the regression
X = ['Pt', 'ScEta', 'Phi', 'rho']  # <<-- list

In [3]:
# Set the quantiles 
quantiles = [ 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9 ]
print "Number of quantiles ", len(quantiles)

Number of quantiles  9


In [4]:
# Initialize the quantile regression object for mc
qr_mc = quantileRegression("mc")

In [ ]:
# This is what you want to correct
print "Load the mc dataframe"
startEvtmc = 2000001 
stopEvtmc  = -1
qr_mc.loadDF("/Users/mauro/CMS/Physics/dataMC/nt/double_ele_spring16v2_sync_v1_mc/",
             "cicNoSigmaIetaIeta/trees/",
             ["DYToEE_powheg_13TeV_EBHighR9", "DYToEE_powheg_13TeV_EBLowR9", "DYToEE_powheg_13TeV_EEHighR9", "DYToEE_powheg_13TeV_EELowR9" ],
             startEvtmc, stopEvtmc, 12345)

Load the mc dataframe
NT branches: 
['rho', 'nvtx', 'leadPt', 'leadScEta', 'leadPhi', 'leadR9', 'leadS4', 'leadSigmaIeIe', 'leadEtaWidth', 'leadPhiWidth', 'leadCovarianceIphiIphi', 'leadSigmaRR', 'subleadPt', 'subleadScEta', 'subleadPhi', 'subleadR9', 'subleadS4', 'subleadSigmaIeIe', 'subleadEtaWidth', 'subleadPhiWidth', 'subleadCovarianceIphiIphi', 'subleadSigmaRR']
Reading trees
Number of events  
rho                          2393830
nvtx                         2393830
leadPt                       2393830
leadScEta                    2393830
leadPhi                      2393830
leadR9                       2393830
leadS4                       2393830
leadSigmaIeIe                2393830
leadEtaWidth                 2393830
leadPhiWidth                 2393830
leadCovarianceIphiIphi       2393830
leadSigmaRR                  2393830
subleadPt                    2393830
subleadScEta                 2393830
subleadPhi                   2393830
subleadR9                    2393830
suble

In [ ]:
#print "Get the corrected data-mc values"
qr_mc.correctAllY(X, ylist, quantiles )

This will take a while...
Loading mc weights for  R9
Loading data weights for  R9
Correcting 

In [ ]:
dfmc = qr_mc.getDF()
# print dfmc

In [ ]:
# now get the data
qr_data = quantileRegression("data")
startEvtdata = 0
stopEvtdata   = -1
print "Load the data dataframe"
qr_data.loadDF("/Users/mauro/CMS/Physics/dataMC/nt/double_ele_spring16v2_sync_v1_ichep/",
               "cicNoSigmaIetaIeta/trees/",
               ["Data_13TeV_EBHighR9", "Data_13TeV_EBLowR9", "Data_13TeV_EEHighR9", "Data_13TeV_EELowR9" ],
               startEvtdata, stopEvtdata)

In [ ]:
dfdata = qr_data.getDF()
#print dfdata

In [ ]:
dfmcNOcorr = dfmc[['index', 'rho', 'nvtx', 'Pt', 'ScEta', 'Phi', 'R9', 'S4', 'SigmaIeIe', 'EtaWidth', 'PhiWidth', 'CovarianceIphiIphi', 'SigmaRR']]
dfmccorr   = dfmc[['index', 'rho', 'nvtx', 'Pt', 'ScEta', 'Phi', 'R9_corr', 'S4_corr', 'SigmaIeIe_corr', 'EtaWidth_corr', 'PhiWidth_corr', 'CovarianceIphiIphi_corr', 'SigmaRR_corr']]

# It needs the same column names to compute the difference between the correlation matrices
dfmccorr.rename(columns={'R9_corr':'R9', 'S4_corr':'S4', 'SigmaIeIe_corr':'SigmaIeIe', 'EtaWidth_corr':'EtaWidth', 'PhiWidth_corr':'PhiWidth', 'CovarianceIphiIphi_corr':'CovarianceIphiIphi', 'SigmaRR_corr':'SigmaRR'}, inplace=True)
# print dfmccorr

In [ ]:
#print dfdata
#print dfmcNOcorr
#print dfmccorr

In [ ]:
correldata     = dfdata.corr()
correlmcNOcorr = dfmcNOcorr.corr()
correlmccorr   = dfmccorr.corr()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
#print correldata
# 'index', 'rho', 'nvtx', 'Pt', 'ScEta', 'Phi', 'R9', 'S4', 'SigmaIeIe', 'EtaWidth', 'PhiWidth', 'CovarianceIphiIphi', 'SigmaRR'
plt.matshow(correldata)
plt.colorbar()

In [ ]:
#print correlmcNOcorr
plt.matshow(correlmcNOcorr)
plt.colorbar()

In [ ]:
#print correlmccorr
plt.matshow(correlmcNOcorr)
plt.colorbar()

In [ ]:
diffCorrelMcNOCorrData = correldata - correlmcNOcorr
#print diffCorrelMcNOCorrData
plt.matshow(diffCorrelMcNOCorrData)
plt.colorbar()

In [ ]:
diffCorrelMcCorrData = correldata - correlmccorr
#print diffCorrelMcCorrData
plt.matshow(diffCorrelMcCorrData)
plt.colorbar()

In [ ]:
test = diffCorrelMcNOCorrData - diffCorrelMcCorrData
plt.matshow(test)
plt.colorbar()

The effect of the corrections (...as expected) has a very small impact.
Need to go more granular

Remember there is no data/mc reweighting done ! ...maybe not needed because we train on pt/eta/phi/rho


This is global. I need more local checks in pt/eta/phi/rho:
- [corr(mccorr)-corr(data)] vs. pt/eta/phi/rho
    With the dataframe mccorr created above and the one on data, 
    select within the dataframe a range in pt/eta/phi/rho and compute 
    [corr(mccorr)-corr(data)] 

In [ ]:
#plt.scatter(dfdata['ScEta'], dfdata['R9'],marker='.', c='green')
#plt.scatter(dfmc['ScEta'], dfmc['R9'],marker='.', c='red')
#plt.scatter(dfmc['ScEta'], dfmc['R9_corr'],marker='.', c='blue')

In [ ]:
#poors man profile plot
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt

def Profile(x,y,nbins,xmin,xmax,ax, color):
    df = DataFrame({'x' : x , 'y' : y})
    
    binedges = xmin + ((xmax-xmin)/nbins) * np.arange(nbins+1)
    df['bin'] = np.digitize(df['x'],binedges)

    # print df
    
    bincenters = xmin + ((xmax-xmin)/nbins)*np.arange(nbins) + ((xmax-xmin)/(2*nbins))
    # print bincenters
    
    ProfileFrame = DataFrame({'bincenters' : bincenters, 'N' : df['bin'].value_counts(sort=False)},index=range(1,nbins+1))
    # ProfileFrame['N'] = ProfileFrame['N'].fillna(1)
    # print ProfileFrame['N']

    bins = ProfileFrame.index.values
    for bin in bins:
        ProfileFrame.ix[bin,'ymean']      = df.ix[df['bin']==bin,'y'].mean()
        ProfileFrame.ix[bin,'yStandDev']  = df.ix[df['bin']==bin,'y'].std()
        ProfileFrame.ix[bin,'yMeanError'] = ProfileFrame.ix[bin,'yStandDev'] / np.sqrt(ProfileFrame.ix[bin,'N'])

    ProfileFrame['ymean']      = ProfileFrame['ymean'].fillna(0)
    ProfileFrame['yMeanError'] = ProfileFrame['yMeanError'].fillna(0)
    #ax.errorbar(ProfileFrame['bincenters'], ProfileFrame['ymean'], yerr=np.array(ProfileFrame['yMeanError']), c=color) #, xerr=(xmax-xmin)/(2*nbins), fmt=None) 
    ax.errorbar(ProfileFrame['bincenters'], ProfileFrame['ymean'], c=color)     
    return ax

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
# loop over all targets ["R9", "S4", "SigmaIeIe", "EtaWidth", "PhiWidth", "CovarianceIphiIphi", "SigmaRR"]
ylist = ["R9", "S4", "SigmaIeIe", "EtaWidth", "PhiWidth", "CovarianceIphiIphi", "SigmaRR"]

for yvar in ylist:
    yvarcorr = yvar+"_corr"

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_ylim(0.6,1.0)
    ax.set_xlabel('ScEta')
    ax.set_ylabel(yvar)
    Profile(dfmc['ScEta']  , dfmc[yvar]     ,100,-2.5,2.5,ax,'green')
    Profile(dfdata['ScEta'], dfdata[yvar]   ,100,-2.5,2.5,ax, 'red')
    Profile(dfmc['ScEta']  , dfmc[yvarcorr],100,-2.5,2.5,ax, 'blue')
    fig.savefig(yvar+"eta.pdf")

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_ylim(0.7,0.9)
    ax.set_xlabel('Pt')
    ax.set_ylabel(yvar)
    Profile(dfmc['Pt']  , dfmc[yvar]     ,100,20.,80.,ax, 'green')
    Profile(dfdata['Pt'], dfdata[yvar]   ,100,20.,80.,ax, 'red')
    Profile(dfmc['Pt']  , dfmc[yvarcorr],100,20.,80.,ax, 'blue')
    fig.savefig(yvar+"pt.pdf")

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_ylim(0.7,0.9)
    ax.set_xlabel('rho')
    ax.set_ylabel(yvar)
    Profile(dfmc['rho']  , dfmc[yvar]     ,100,0.,40.,ax, 'green')
    Profile(dfdata['rho'], dfdata[yvar]   ,100,0.,40.,ax, 'red')
    Profile(dfmc['rho']  , dfmc[yvarcorr],100,0.,40.,ax, 'blue')
    fig.savefig(yvar+"rho.pdf")

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_ylim(0.7,0.9)
    ax.set_xlabel('nvtx')
    ax.set_ylabel(yvar)
    Profile(dfmc['nvtx']  , dfmc[yvar]     ,41,0.,40.,ax, 'green')
    Profile(dfdata['nvtx'], dfdata[yvar]   ,41,0.,40.,ax, 'red')
    Profile(dfmc['nvtx']  , dfmc[yvarcorr],41,0.,40.,ax, 'blue')
    fig.savefig(yvar+"nvtx.pdf")
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_ylim(0.75,0.95)
    ax.set_xlabel('Phi')
    ax.set_ylabel(yvar)
    Profile(dfmc['Phi']  , dfmc[yvar]     ,50,-3.14,3.14,ax, 'green')
    Profile(dfdata['Phi'], dfdata[yvar]   ,50,-3.14,3.14,ax, 'red')
    Profile(dfmc['Phi']  , dfmc[yvarcorr],50,-3.14,3.14,ax, 'blue')
    fig.savefig(yvar+"phi.pdf")

In [ ]:
# Region definitions

eta = ['EB','EE']
pT  = ['37']
rho = ['12']

mcBinNames = []
dataBinNames = []
binFileNames = []
plotTitles = []

for ieta in eta:
    etabin = ieta
    etacut = ''
    if ieta == 'EB': etacut = '(abs(ScEta) < 1.5)'
    if ieta == 'EE': etacut = '(abs(ScEta) > 1.5)'
    # print etabin

    for ipT in range(0,len(pT)+1):
        ptbin = ''
        ptcut = ''
        # print 'ipT', ipT
        if ipT == 0:
            ptbin = 'pTlt'+pT[ipT]
            ptcut = '(Pt <' + pT[ipT] +')'
        elif ipT < len(pT):
            ptbin = 'pT'+pT[ipT-1]+pT[ipT]
            ptcut = '('+pT[ipT-1] +' <= Pt) & ( Pt < '+pT[ipT] +')'
        elif ipT == len(pT):
            ptbin = 'pTgt'+pT[ipT-1]
            ptcut = '( Pt >=' + pT[ipT-1] +')'
        # print "    ", ptbin

        for irho in range(0,len(rho)+1):
            rhobin = ''
            rhocut = ''
            # print 'irho', irho
            if irho == 0:
                rhobin = 'rholt'+rho[irho]
                rhocut = '(rho <' + rho[irho] +')'
            elif irho < len(rho):
                rhobin = 'rho'+rho[irho-1]+rho[irho]
                rhocut = '('+rho[irho-1] +' <= rho) & ( rho < '+rho[irho] +')'
            elif irho == len(rho):
                rhobin = 'rhogt'+rho[irho-1]
                rhocut = '( rho >=' + rho[irho-1] +')'
            # print "        ", rhobin
            
            binFileNames.append('./plots/'+etabin + ptbin + rhobin)
            mcBinNames.append('dfmc.query(\'' + etacut + ' & ' + ptcut + ' & ' + rhocut + '\')')
            dataBinNames.append('dfdata.query(\'' + etacut + ' & ' + ptcut + ' & ' + rhocut + '\')')
            plotTitles.append(etacut + ' & ' + ptcut + ' & ' + rhocut)

#for i in mcBinNames:
#    print i
#for i in dataBinNames:
#    print i
#for i in plotTitles:
#    print i
#for i in binFileNames:
#    print i



In [ ]:
# loop over all targets ["R9", "S4", "SigmaIeIe", "EtaWidth", "PhiWidth", "CovarianceIphiIphi", "SigmaRR"]
ylist = ["R9", "S4", "SigmaIeIe", "EtaWidth", "PhiWidth", "CovarianceIphiIphi", "SigmaRR"]

for yvar in ylist:
    yvarcorr = yvar+"_corr"
    for i in range(0,len(plotTitles)):
        print mcBinNames[i]
        print dataBinNames[i]
        mcbin   = eval(mcBinNames[i])
        databin = eval(dataBinNames[i])

            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.set_ylim(0.6,1.0)
            ax.set_title(plotTitles[i])
            ax.set_xlabel('ScEta')
            ax.set_ylabel(yvar)
            Profile(mcbin['ScEta']  , mcbin[yvar]     ,100,-2.5,2.5,ax,'green')
            Profile(databin['ScEta'], databin[yvar]   ,100,-2.5,2.5,ax, 'red')
            Profile(mcbin['ScEta']  , mcbin[yvarcorr],100,-2.5,2.5,ax, 'blue')
            fig.savefig(binFileNames[i]+'_'+yvar+"eta.pdf")

            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.set_ylim(0.7,0.9)
            ax.set_title(plotTitles[i])
            ax.set_xlabel('Pt')
            ax.set_ylabel(yvar)
            Profile(mcbin['Pt']  , mcbin[yvar]     ,100,20.,80.,ax, 'green')
            Profile(databin['Pt'], databin[yvar]   ,100,20.,80.,ax, 'red')
            Profile(mcbin['Pt']  , mcbin[yvarcorr],100,20.,80.,ax, 'blue')
            fig.savefig(binFileNames[i]+'_'+yvar+"pt.pdf")

            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.set_ylim(0.7,0.9)
            ax.set_title(plotTitles[i])
            ax.set_xlabel('rho')
            ax.set_ylabel(yvar)
            Profile(mcbin['rho']  , mcbin[yvar]     ,100,0.,40.,ax, 'green')
            Profile(databin['rho'], databin[yvar]   ,100,0.,40.,ax, 'red')
            Profile(mcbin['rho']  , mcbin[yvarcorr],100,0.,40.,ax, 'blue')
            fig.savefig(binFileNames[i]+'_'+yvar+"rho.pdf")

            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.set_ylim(0.7,0.9)
            ax.set_title(plotTitles[i])
            ax.set_xlabel('nvtx')
            ax.set_ylabel(yvar)
            Profile(mcbin['nvtx']  , mcbin[yvar]     ,41,0.,40.,ax, 'green')
            Profile(databin['nvtx'], databin[yvar]   ,41,0.,40.,ax, 'red')
            Profile(mcbin['nvtx']  , mcbin[yvarcorr],41,0.,40.,ax, 'blue')
            fig.savefig(binFileNames[i]+'_'+yvar+"nvtx.pdf")

            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.set_ylim(0.75,0.95)
            ax.set_title(plotTitles[i])
            ax.set_xlabel('Phi')
            ax.set_ylabel(yvar)
            Profile(mcbin['Phi']  , mcbin[yvar]     ,50,-3.14,3.14,ax, 'green')
            Profile(databin['Phi'], databin[yvar]   ,50,-3.14,3.14,ax, 'red')
            Profile(mcbin['Phi']  , mcbin[yvarcorr],50,-3.14,3.14,ax, 'blue')
            fig.savefig(binFileNames[i]+'_'+yvar+"phi.pdf")